In [1]:
# Import libraries
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import matplotlib.dates as mdates
import statsmodels.api as sm
from scipy.stats import pearsonr
import itertools
from joblib import Parallel, delayed
import multiprocessing
import datetime

In [2]:
# establish connection with wrds
conn = wrds.Connection()

Enter your WRDS username [komalniraula]: kn2505
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
# Collecting data from 2012 to calculate features like rolling returns and stability
# The final report is based on trading(price) data 2015-2024 
# But company financials from 2014 3rd quarter to 2024 second quarter reports
years = list(range(2012, 2025)) 
df_list = []

for i, year in enumerate(years):
    print(f"Loading {year} ({i+1}/{len(years)})...")
    df_year = conn.raw_sql(f"""
        SELECT *
        FROM comp_na_daily_all.fundq
        WHERE indfmt = 'INDL'
          AND datafmt = 'STD'
          AND popsrc = 'D'
          AND consol = 'C'
          AND EXTRACT(YEAR FROM datadate) = {year}
    """, date_cols=['datadate'])
    df_list.append(df_year)

df_comp_main = pd.concat(df_list, ignore_index=True)

Loading 2012 (1/13)...
Loading 2013 (2/13)...
Loading 2014 (3/13)...
Loading 2015 (4/13)...
Loading 2016 (5/13)...
Loading 2017 (6/13)...
Loading 2018 (7/13)...
Loading 2019 (8/13)...
Loading 2020 (9/13)...
Loading 2021 (10/13)...
Loading 2022 (11/13)...
Loading 2023 (12/13)...
Loading 2024 (13/13)...


/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/2913834407.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comp_main = pd.concat(df_list, ignore_index=True)


In [15]:
# creating dublicate to work on so shouldn't download entire data if wann'a start again
df_comp = df_comp_main.copy() 

In [16]:
# Looking at alphabet data (gvkey = 160329) to get overview of whats there
df_comp_ = df_comp[['gvkey', 'rdq', 'fyearq', 'fqtr', 'cusip', 'fyr', 'prchq', 'prclq', 'cdvcy', 'dvy', 'dvpspq', 'optdrq', 
                    'datafqtr', 'cdvcy']]
df_comp_= df_comp_[df_comp_['gvkey'] == '160329']
df_comp_

,gvkey,rdq,fyearq,fqtr,cusip,fyr,prchq,prclq,cdvcy,dvy,dvpspq,optdrq,datafqtr,cdvcy
28979,160329,2012-04-12,2012,1,02079K305,12,670.25,564.55,<NA>,0.0,0.0,<NA>,2012Q1,<NA>
28980,160329,2012-07-19,2012,2,02079K305,12,653.14,556.5201,<NA>,0.0,0.0,0.0,2012Q2,<NA>
28981,160329,2012-10-18,2012,3,02079K305,12,764.89,562.09,<NA>,0.0,0.0,<NA>,2012Q3,<NA>
28982,160329,2013-01-22,2012,4,02079K305,12,774.38,636.0,<NA>,0.0,0.0,0.0,2012Q4,<NA>
77022,160329,2013-04-18,2013,1,02079K305,12,844.0,695.52,<NA>,0.0,0.0,0.0,2013Q1,<NA>
77023,160329,2013-07-18,2013,2,02079K305,12,920.6,761.2601,<NA>,0.0,0.0,<NA>,2013Q2,<NA>
77024,160329,2013-10-17,2013,3,02079K305,12,928.0,845.56,<NA>,0.0,0.0,<NA>,2013Q3,<NA>
77025,160329,2014-01-30,2013,4,02079K305,12,1121.0,842.98,<NA>,0.0,0.0,0.0,2013Q4,<NA>
124966,160329,2014-04-16,2014,1,02079K305,12,1228.88,1082.27,<NA>,0.0,0.0,<NA>,2014Q1,<NA>
124967,160329,2014-07-17,2014,2,02079K305,12,1144.8,511.0,<NA>,0.0,0.0,<NA>,2014Q2,<NA>


### **Financial ratios for knn-clustering**

In [17]:
df_comp = df_comp.sort_values(by=['gvkey', 'datadate'])

# Calculate quarterly CapEx from year-to-date capxy
df_comp['capxq'] = df_comp.groupby('gvkey')['capxy'].diff()
df_comp['fqtr'] = df_comp['fqtr'].fillna(df_comp['datadate'].dt.quarter)
df_comp.loc[df_comp['fqtr'] == 1, 'capxq'] = df_comp['capxy']

# Core financial ratios
df_comp['roa'] = df_comp['niq'] / df_comp['atq']
df_comp['operating_margin'] = df_comp['oiadpq'] / df_comp['saleq']
df_comp['gross_margin'] = (df_comp['saleq'] - df_comp['cogsq']) / df_comp['saleq']
df_comp['revenue_growth'] = df_comp.groupby('gvkey')['saleq'].pct_change()
df_comp['r_and_d_intensity'] = df_comp['xrdq'] / df_comp['atq']
df_comp['capex_intensity'] = df_comp['capxq'] / df_comp['atq']

# Accruals (Sloan)
df_comp['atq_lag'] = df_comp.groupby('gvkey')['atq'].shift(1)
df_comp['d_actq'] = df_comp.groupby('gvkey')['actq'].diff()
df_comp['d_cheq'] = df_comp.groupby('gvkey')['cheq'].diff()
df_comp['d_lctq'] = df_comp.groupby('gvkey')['lctq'].diff()
df_comp['d_dd1q'] = df_comp.groupby('gvkey')['dd1q'].diff()
df_comp['d_txpq'] = df_comp.groupby('gvkey')['txpq'].diff()

df_comp['sloan_accruals'] = (
    (df_comp['d_actq'] - df_comp['d_cheq']) -
    (df_comp['d_lctq'] - df_comp['d_dd1q'] - df_comp['d_txpq']) -
    df_comp['dpq']
) / df_comp['atq_lag']

# Rolling 8-quarter volatility
df_comp['roa_stability'] = df_comp.groupby('gvkey')['roa'].rolling(8).std().reset_index(level=0, drop=True)
df_comp['revenue_growth_stability'] = df_comp.groupby('gvkey')['revenue_growth'].rolling(8).std().reset_index(level=0, drop=True)

# Winsorization function
def winsorize(series, lower=0.01, upper=0.99):
    return series.clip(lower=series.quantile(lower), upper=series.quantile(upper))

ratio_columns = [
    'roa', 'operating_margin', 'gross_margin', 'sloan_accruals',
    'revenue_growth', 'r_and_d_intensity', 'capex_intensity',
    'roa_stability', 'revenue_growth_stability'
]

for col in ratio_columns:
    df_comp[col] = winsorize(df_comp[col])

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/1156743967.py:12: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_comp['revenue_growth'] = df_comp.groupby('gvkey')['saleq'].pct_change()
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [18]:
# Keep the row with fqtr=1 when there are duplicate gvkey+datadate
df_comp = df_comp.sort_values(['gvkey', 'datadate', 'fqtr'])  # ascending: fqtr=1 will be last
df_comp = df_comp.drop_duplicates(subset=['gvkey', 'datadate'], keep='last')

In [19]:
df_comp['datadate'] = pd.to_datetime(df_comp['datadate'])
df_comp['year']=df_comp['datadate'].dt.year
df_comp.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,atq_lag,d_actq,d_cheq,d_lctq,d_dd1q,d_txpq,sloan_accruals,roa_stability,revenue_growth_stability,year
0,001004,2012-02-29,2011,3,5,INDL,C,D,STD,AIR,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,2012
1,001004,2012-05-31,2011,4,5,INDL,C,D,STD,AIR,...,2220.293,-2.117,8.426,-87.76,102.08,0.0,0.067813,NaN,NaN,2012
2,001004,2012-08-31,2012,1,5,INDL,C,D,STD,AIR,...,2195.653,-29.672,-0.02,-56.626,-14.637,0.0,-0.006314,NaN,NaN,2012
3,001004,2012-11-30,2012,2,5,INDL,C,D,STD,AIR,...,2170.3,13.2,1.5,3.1,-1.6,0.0,-0.009261,NaN,NaN,2012
47282,001004,2013-02-28,2012,3,5,INDL,C,D,STD,AIR,...,2189.3,29.3,-21.8,-56.9,-84.4,0.0,-0.001873,NaN,NaN,2013


### **Calculating book to market b/m**

In [20]:
# Preferred Stock: PSTKRQ → PSTKNQ → 0
df_comp['ps'] = df_comp['pstkrq'].fillna(df_comp['pstknq']).fillna(0)

# Stockholders’ Equity (SEQ): SEQQ → ATQ - LTQ → CEQQ + PSTKQ + TXDITCQ
df_comp['seqq'] = df_comp['seqq'].fillna(df_comp['atq'] - df_comp['ltq'])
df_comp['seqq'] = df_comp['seqq'].fillna(df_comp['ceqq'] + df_comp['pstkq'] + df_comp['txditcq'])

# Book Equity = SEQ - Preferred Stock
df_comp['be'] = df_comp['seqq'] - df_comp['ps']
df_comp['be'] = df_comp['be'].where(df_comp['be'] > 0)

In [21]:
# sort each firm's accounting records in chronological order
df_comp = df_comp.sort_values(by=['gvkey', 'datadate'])

# add a counter to track how many quarter of data exist per firm
df_comp['count'] = df_comp.groupby('gvkey').cumcount()

# retain only relevant columns for merging and future use
df_comp = df_comp[['gvkey', 'datadate', 'year', 'be', 'count'] + ratio_columns]
df_comp

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,r_and_d_intensity,capex_intensity,roa_stability,revenue_growth_stability
0,001004,2012-02-29,2012,890.08,0,0.009306,0.066672,0.198164,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1,001004,2012-05-31,2012,864.649,1,0.005866,0.069975,0.208203,0.067813,0.072354,<NA>,0.012987,NaN,NaN
2,001004,2012-08-31,2012,880.6,2,0.008386,0.067212,0.211626,-0.006314,-0.039009,<NA>,0.005068,NaN,NaN
3,001004,2012-11-30,2012,902.8,3,0.00813,0.071373,0.223284,-0.009261,-0.068483,<NA>,0.0037,NaN,NaN
47282,001004,2013-02-28,2013,924.0,4,0.0083,0.048827,0.199731,-0.001873,0.014431,<NA>,0.002165,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566455,356687,2023-12-31,2023,87.766,7,-0.060509,-inf,NaN,<NA>,NaN,<NA>,0.102836,0.819372,NaN
615195,356687,2024-03-31,2024,150.626,8,-0.027518,-inf,NaN,0.029428,NaN,<NA>,0.039107,0.068628,NaN
615196,356687,2024-06-30,2024,209.337,9,-0.008274,-inf,NaN,-0.009785,NaN,<NA>,0.042664,0.033313,NaN
615197,356687,2024-09-30,2024,209.318,10,-0.004379,-inf,NaN,-0.002431,NaN,<NA>,0.03859,0.029810,NaN


In [22]:
df_comp.isnull().sum()

gvkey                            0
datadate                         0
year                             0
be                          253667
count                            0
roa                         203276
operating_margin            197677
gross_margin                192555
sloan_accruals              357273
revenue_growth              190229
r_and_d_intensity           450086
capex_intensity             239093
roa_stability               312885
revenue_growth_stability    342824
dtype: int64

In [23]:
# CRSP Block      
df_crsp = conn.raw_sql("""
    SELECT a.permno, a.permco, a.date, b.shrcd, b.exchcd, b.ticker, b.cusip,
    a.ret, a.retx, a.shrout, a.prc
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
      ON a.permno = b.permno
      AND b.namedt <= a.date
      AND a.date <= b.nameendt
      WHERE a.date BETWEEN '01/01/1948' AND '12/31/2024'
      AND b.exchcd BETWEEN 1 AND 3  -- NYSE, AMEX, NASDAQ
      AND b.shrcd IN (10, 11)       -- Common stocks only
""", date_cols=['date'])
#df_crsp = pd.read_csv("crsp.csv")
df_crsp.head()

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,shrout,prc
0,10000,7952,1986-01-31,10,3,OMFGA,68391610,<NA>,<NA>,3680.0,-4.375
1,10000,7952,1986-02-28,10,3,OMFGA,68391610,-0.257143,-0.257143,3680.0,-3.25
2,10000,7952,1986-03-31,10,3,OMFGA,68391610,0.365385,0.365385,3680.0,-4.4375
3,10000,7952,1986-04-30,10,3,OMFGA,68391610,-0.098592,-0.098592,3793.0,-4.0
4,10000,7952,1986-05-30,10,3,OMFGA,68391610,-0.222656,-0.222656,3793.0,-3.10938


In [24]:
# change variable format to int
df_crsp[['permco','permno','shrcd','exchcd']]=df_crsp[['permco','permno','shrcd','exchcd']].astype(int)

df_crsp['date'] = pd.to_datetime(df_crsp['date'])

# roll forward to the end of the month, if current date is not the end of the month
df_crsp['jdate']=df_crsp['date']+MonthEnd(0) 

In [25]:
# fill missing return values with 0
df_crsp['ret'] = df_crsp['ret'].fillna(0)   # total return (includes delisting, as provided by CRSP)
df_crsp['retx'] = df_crsp['retx'].fillna(0) # return excluding delisting

# calculate market equity (in $ thousands)
df_crsp['ME'] = df_crsp['prc'].abs() * df_crsp['shrout']  # prc is negative if bid quote, so take abs()

# drop columns no longer needed
df_crsp = df_crsp.drop(['prc', 'shrout'], axis=1)

# sort by date, firm ID (permco), and market equity
df_crsp = df_crsp.sort_values(by=['date', 'permco', 'ME'])

# view cleaned data
df_crsp.head()

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,jdate,ME
94388,17670,74,1948-01-31,10,1,<NA>,02342410,-0.045064,-0.062232,1948-01-31,6828.125
155114,18702,267,1948-01-31,10,1,<NA>,03333010,-0.02381,-0.02381,1948-01-31,25922.25
262852,20714,584,1948-01-31,10,1,<NA>,09788010,-0.058824,-0.058824,1948-01-31,40512.0
167367,11287,921,1948-01-31,10,1,<NA>,17286410,0.054217,0.054217,1948-01-31,38915.625
286474,21151,994,1948-01-31,10,1,<NA>,23111110,0.007692,-0.007692,1948-01-31,14399.625


In [26]:
# sum of ME across all permnos belonging to the same permco on a given date
crsp_sumME = df_crsp.groupby(['jdate', 'permco'])['ME'].sum().reset_index()

# find the largest single ME (security) per permco/date
crsp_maxME = df_crsp.groupby(['jdate', 'permco'])['ME'].max().reset_index()

# keep only the permno with the largest ME per firm/date
df_crsp1 = pd.merge(df_crsp, crsp_maxME, how='inner', on=['jdate', 'permco', 'ME'])

# drop ME (security-level) and replace it with firm-level total ME
df_crsp1 = df_crsp1.drop(['ME'], axis=1)

# add back the total firm-level ME
df_crsp2 = pd.merge(df_crsp1, crsp_sumME, how='inner', on=['jdate', 'permco'])  # New 'ME' is total ME

# sort and drop duplicates (in case of multiple securities with identical ME)
df_crsp2 = df_crsp2.sort_values(by=['permno', 'jdate']).drop_duplicates()

df_crsp2.head()

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,jdate,ME
1238014,10000,7952,1986-01-31,10,3,OMFGA,68391610,0.0,0.0,1986-01-31,16100.0
1243659,10000,7952,1986-02-28,10,3,OMFGA,68391610,-0.257143,-0.257143,1986-02-28,11960.0
1249305,10000,7952,1986-03-31,10,3,OMFGA,68391610,0.365385,0.365385,1986-03-31,16330.0
1254957,10000,7952,1986-04-30,10,3,OMFGA,68391610,-0.098592,-0.098592,1986-04-30,15172.0
1260614,10000,7952,1986-05-30,10,3,OMFGA,68391610,-0.222656,-0.222656,1986-05-31,11793.87834


In [27]:
# extract year and month from jdate
df_crsp2['year'] = df_crsp2['jdate'].dt.year
df_crsp2['month'] = df_crsp2['jdate'].dt.month

In [28]:
def assign_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

df_crsp2['qtr'] = df_crsp2['month'].apply(assign_quarter)

df_crsp2['qkey'] = df_crsp2['year'].astype(str) + '-' + df_crsp2['qtr']

df_crsp2['1+retx'] = 1 + df_crsp2['retx'].fillna(0)

df_crsp2['cumretx_q'] = df_crsp2.groupby(['permno', 'qkey'])['1+retx'].cumprod()
df_crsp2

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,jdate,ME,year,month,qtr,qkey,1+retx,cumretx_q
1238014,10000,7952,1986-01-31,10,3,OMFGA,68391610,0.0,0.0,1986-01-31,16100.0,1986,1,Q1,1986-Q1,1.0,1.0
1243659,10000,7952,1986-02-28,10,3,OMFGA,68391610,-0.257143,-0.257143,1986-02-28,11960.0,1986,2,Q1,1986-Q1,0.742857,0.742857
1249305,10000,7952,1986-03-31,10,3,OMFGA,68391610,0.365385,0.365385,1986-03-31,16330.0,1986,3,Q1,1986-Q1,1.365385,1.014286
1254957,10000,7952,1986-04-30,10,3,OMFGA,68391610,-0.098592,-0.098592,1986-04-30,15172.0,1986,4,Q2,1986-Q2,0.901408,0.901408
1260614,10000,7952,1986-05-30,10,3,OMFGA,68391610,-0.222656,-0.222656,1986-05-31,11793.87834,1986,5,Q2,1986-Q2,0.777344,0.700704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559840,93436,53453,2024-08-30,11,3,TSLA,88160R10,-0.077391,-0.077391,2024-08-31,684004370.4,2024,8,Q3,2024-Q3,0.922609,1.082018
3563671,93436,53453,2024-09-30,11,3,TSLA,88160R10,0.221942,0.221942,2024-09-30,839047410.0,2024,9,Q3,2024-Q3,1.221942,1.322164
3567481,93436,53453,2024-10-31,11,3,TSLA,88160R10,-0.045025,-0.045025,2024-10-31,802033523.1006,2024,10,Q4,2024-Q4,0.954975,0.954975
3571279,93436,53453,2024-11-29,11,3,TSLA,88160R10,0.381469,0.381469,2024-11-30,1107984309.6,2024,11,Q4,2024-Q4,1.381469,1.319268


#### Linking compustat with crsp

In [29]:
# CCM Block
df_ccm = conn.raw_sql("""
    SELECT gvkey, lpermno AS permno, linktype, linkprim, 
           linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE SUBSTR(linktype, 1, 1) = 'L'       -- Only active "link" types (e.g., LU, LC)
      AND (linkprim = 'C' OR linkprim = 'P') -- Primary links: 'C' (most reliable), 'P' (used if 'C' missing)
""", date_cols=['linkdt', 'linkenddt'])
#df_ccm = pd.read_csv("ccm.csv")
df_ccm.head()

,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30
1,001001,10015.0,LU,P,1983-09-20,1986-07-31
2,001002,10023.0,LC,C,1972-12-14,1973-06-05
3,001003,10031.0,LU,C,1983-12-07,1989-08-16
4,001004,54594.0,LU,P,1972-04-24,NaT


In [30]:
df_ccm['linkenddt'] = pd.to_datetime(df_ccm['linkenddt'])

# if linkenddt is missing then set to today date
df_ccm['linkenddt']=df_ccm['linkenddt'].fillna(pd.to_datetime('today')) # setting those to today's date makes sure the link is considered valid when merging historical Compustat and CRSP data

# associate each Compustat firm (gvkey) with its corresponding CRSP stock (permno) to eventually match with stock return data
df_ccm1=pd.merge(df_comp,
                 df_ccm,how='left',on=['gvkey']) # keeps all compustat records and attaches CRSP info if available

#Convert datadate to datetime if not already
df_ccm1['datadate'] = pd.to_datetime(df_ccm1['datadate'])

# Create jdate as the calendar month-end date
df_ccm1['jdate'] = df_ccm1['datadate'] + pd.offsets.MonthEnd(0)

# extract year and month from datadate
df_ccm1['year'] = df_ccm1['datadate'].dt.year
df_ccm1['month'] = df_ccm1['datadate'].dt.month
df_ccm1

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,capex_intensity,roa_stability,revenue_growth_stability,permno,linktype,linkprim,linkdt,linkenddt,jdate,month
0,001004,2012-02-29,2012,890.08,0,0.009306,0.066672,0.198164,<NA>,<NA>,...,<NA>,NaN,NaN,54594.0,LU,P,1972-04-24,2025-05-15 02:17:36.870812,2012-02-29,2
1,001004,2012-05-31,2012,864.649,1,0.005866,0.069975,0.208203,0.067813,0.072354,...,0.012987,NaN,NaN,54594.0,LU,P,1972-04-24,2025-05-15 02:17:36.870812,2012-05-31,5
2,001004,2012-08-31,2012,880.6,2,0.008386,0.067212,0.211626,-0.006314,-0.039009,...,0.005068,NaN,NaN,54594.0,LU,P,1972-04-24,2025-05-15 02:17:36.870812,2012-08-31,8
3,001004,2012-11-30,2012,902.8,3,0.00813,0.071373,0.223284,-0.009261,-0.068483,...,0.0037,NaN,NaN,54594.0,LU,P,1972-04-24,2025-05-15 02:17:36.870812,2012-11-30,11
4,001004,2013-02-28,2013,924.0,4,0.0083,0.048827,0.199731,-0.001873,0.014431,...,0.002165,NaN,NaN,54594.0,LU,P,1972-04-24,2025-05-15 02:17:36.870812,2013-02-28,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679402,356687,2023-12-31,2023,87.766,7,-0.060509,-inf,NaN,<NA>,NaN,...,0.102836,0.819372,NaN,<NA>,<NA>,<NA>,NaT,NaT,2023-12-31,12
679403,356687,2024-03-31,2024,150.626,8,-0.027518,-inf,NaN,0.029428,NaN,...,0.039107,0.068628,NaN,<NA>,<NA>,<NA>,NaT,NaT,2024-03-31,3
679404,356687,2024-06-30,2024,209.337,9,-0.008274,-inf,NaN,-0.009785,NaN,...,0.042664,0.033313,NaN,<NA>,<NA>,<NA>,NaT,NaT,2024-06-30,6
679405,356687,2024-09-30,2024,209.318,10,-0.004379,-inf,NaN,-0.002431,NaN,...,0.03859,0.029810,NaN,<NA>,<NA>,<NA>,NaT,NaT,2024-09-30,9


In [31]:
# only keep rows where jdate (portfolio assignment date) falls within the link window (linkdt to linkenddt)
df_ccm2 = df_ccm1[
    (df_ccm1['jdate'] >= df_ccm1['linkdt']) &
    (df_ccm1['jdate'] <= df_ccm1['linkenddt'])
]

In [32]:
invalid_months = df_ccm2[~df_ccm2['month'].isin([3, 6, 9, 12])]
print(f"Number of non-quarter-end rows: {len(invalid_months)}")

Number of non-quarter-end rows: 36314


In [33]:
# map quarter to month
def map_to_quarter_end(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'  # 10, 11, 12 → 12

df_ccm2['quarter'] = df_ccm2['month'].apply(map_to_quarter_end)

duplicates = df_ccm2[df_ccm2.duplicated(subset=['permno', 'month', 'year'], keep=False)]
print(duplicates.sort_values(by=['permno', 'year', 'month']))

        gvkey   datadate  year       be  count       roa  operating_margin  \
74633  011550 2012-03-31  2012   4081.9      0  0.012452          0.248237   
88017  013353 2012-03-31  2012   3278.8      0   0.00993           0.18202   
74634  011550 2012-06-30  2012   4111.5      1   0.00859           0.23563   
88019  013353 2012-06-30  2012   3317.6      1   0.00705          0.157388   
74635  011550 2012-09-30  2012   4177.6      2  0.011283          0.269989   
...       ...        ...   ...      ...    ...       ...               ...   
88083  013353 2020-06-30  2020   3726.3     33  0.005309          0.245094   
74667  011550 2020-09-30  2020  10481.1     34  0.007541          0.224228   
88085  013353 2020-09-30  2020   3846.0     34   0.00856          0.271717   
74668  011550 2020-12-31  2020  10500.1     35  0.006463          0.191631   
88087  013353 2020-12-31  2020   3692.6     35  0.004557          0.218202   

       gross_margin  sloan_accruals  revenue_growth  ...  roa_s

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/3348192014.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ccm2['quarter'] = df_ccm2['month'].apply(map_to_quarter_end)


In [34]:
duplicates = df_ccm2[df_ccm2.duplicated(subset=['permno', 'month', 'year'], keep=False)]
print(duplicates.sort_values(by=['permno', 'year', 'month']))

        gvkey   datadate  year       be  count       roa  operating_margin  \
74633  011550 2012-03-31  2012   4081.9      0  0.012452          0.248237   
88017  013353 2012-03-31  2012   3278.8      0   0.00993           0.18202   
74634  011550 2012-06-30  2012   4111.5      1   0.00859           0.23563   
88019  013353 2012-06-30  2012   3317.6      1   0.00705          0.157388   
74635  011550 2012-09-30  2012   4177.6      2  0.011283          0.269989   
...       ...        ...   ...      ...    ...       ...               ...   
88083  013353 2020-06-30  2020   3726.3     33  0.005309          0.245094   
74667  011550 2020-09-30  2020  10481.1     34  0.007541          0.224228   
88085  013353 2020-09-30  2020   3846.0     34   0.00856          0.271717   
74668  011550 2020-12-31  2020  10500.1     35  0.006463          0.191631   
88087  013353 2020-12-31  2020   3692.6     35  0.004557          0.218202   

       gross_margin  sloan_accruals  revenue_growth  ...  roa_s

In [35]:
# Extract non-duplicate rows
non_duplicates = df_ccm2.drop_duplicates(subset=['permno', 'month', 'year'], keep=False)

# Shuffle duplicates so we can select randomly when needed
duplicates = duplicates.sample(frac=1, random_state=42)

# Apply logic to resolve duplicates
def resolve_duplicates(group):
    non_nan = group[group['be'].notna()]
    if len(non_nan) == 1:
        return non_nan
    else:
        return group.iloc[[0]]  # randomly keep one

deduped = duplicates.groupby(['permno', 'month', 'year'], group_keys=False).apply(resolve_duplicates)

# Combine back with the non-duplicates
df_ccm1_cleaned = pd.concat([non_duplicates, deduped], ignore_index=True)

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/1379524675.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  deduped = duplicates.groupby(['permno', 'month', 'year'], group_keys=False).apply(resolve_duplicates)


In [36]:
duplicates = df_ccm1_cleaned[df_ccm1_cleaned.duplicated(subset=['permno', 'month', 'year'], keep=False)]
print(duplicates.sort_values(by=['permno', 'year', 'month']))

Empty DataFrame
Columns: [gvkey, datadate, year, be, count, roa, operating_margin, gross_margin, sloan_accruals, revenue_growth, r_and_d_intensity, capex_intensity, roa_stability, revenue_growth_stability, permno, linktype, linkprim, linkdt, linkenddt, jdate, month, quarter]
Index: []

[0 rows x 22 columns]


In [37]:
df_merged = pd.merge(
    df_ccm1_cleaned,   # your final Compustat data with jdate
    df_crsp2,          # your CRSP data with jdate
    on=['permno', 'jdate', 'year', 'month'],
    how='left'         # keep all Compustat rows, match CRSP when available
)

# BE is in millions, ME is in thousands → multiply BE by 1000 to match units
df_merged['beme'] = df_merged['be'] * 1000 / df_merged['ME']
df_merged

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
0,001004,2012-02-29,2012,890.08,0,0.009306,0.066672,0.198164,<NA>,<NA>,...,AIR,00036110,0.040585,0.040585,888240.15,Q1,2012-Q1,1.040585,1.150235,1.002071
1,001004,2012-05-31,2012,864.649,1,0.005866,0.069975,0.208203,0.067813,0.072354,...,AIR,00036110,-0.220065,-0.220065,485470.4,Q2,2012-Q2,0.779935,0.660273,1.781054
2,001004,2012-08-31,2012,880.6,2,0.008386,0.067212,0.211626,-0.006314,-0.039009,...,AIR,00036110,0.04715,0.04715,597684.96,Q3,2012-Q3,1.04715,1.103857,1.473351
3,001004,2012-11-30,2012,902.8,3,0.00813,0.071373,0.223284,-0.009261,-0.068483,...,AIR,00036110,0.017893,0.017893,613478.4,Q4,2012-Q4,1.017893,0.935445,1.471608
4,001004,2013-02-28,2013,924.0,4,0.0083,0.048827,0.199731,-0.001873,0.014431,...,AIR,00036110,-0.067905,-0.067905,697669.56,Q1,2013-Q1,0.932095,0.940578,1.324409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394126,013353,2016-12-31,2016,3524.2,19,0.00445,0.114264,0.204082,<NA>,-0.105001,...,WEC,92939U10,0.047135,0.047135,18510937.05,Q4,2016-Q4,1.047135,0.97946,0.190385
394127,013353,2017-12-31,2017,3414.3,23,0.005289,0.143434,0.232536,<NA>,-0.003497,...,WEC,92939U10,-0.044035,-0.044035,20963713.68,Q4,2017-Q4,0.955965,1.05814,0.162867
394128,011550,2018-12-31,2018,9819.3,27,0.006133,0.139542,0.244366,<NA>,0.263491,...,WEC,92939U10,-0.044426,-0.044426,21853330.76,Q4,2018-Q4,0.955574,1.037448,0.449327
394129,011550,2019-12-31,2019,10143.8,31,0.006987,0.186444,0.307728,<NA>,0.211132,...,WEC,92939U10,0.040384,0.040384,29092662.28,Q4,2019-Q4,1.040384,0.969821,0.348672


In [38]:
df_merged.isnull().sum()

gvkey                            0
datadate                         0
year                             0
be                          151163
count                            0
roa                         137230
operating_margin            139160
gross_margin                137264
sloan_accruals              240571
revenue_growth              137568
r_and_d_intensity           270241
capex_intensity             152543
roa_stability               193539
revenue_growth_stability    196507
permno                           0
linktype                         0
linkprim                         0
linkdt                           0
linkenddt                        0
jdate                            0
month                            0
quarter                          0
permco                      203816
date                        203816
shrcd                       203816
exchcd                      203816
ticker                      203816
cusip                       203816
ret                 

In [39]:
print("Rows with missing ME:", df_merged['ME'].isna().sum())
print("Total rows:", len(df_merged))

Rows with missing ME: 203816
Total rows: 394131


In [40]:
missing_permnos = df_merged[df_merged['ME'].isna()]['permno'].unique()
print(f"# permnos not matched: {len(missing_permnos)}")
print("Example permnos:", missing_permnos[:5])

# permnos not matched: 9081
Example permnos: <FloatingArray>
[26649.0, 10065.0, 11077.0, 78223.0, 49330.0]
Length: 5, dtype: Float64


In [41]:
# For df_ccm1_cleaned (Compustat side)
num_nan_permno_ccm = df_ccm1_cleaned['permno'].isna().sum()
print(f"NaN permno in df_ccm1_cleaned: {num_nan_permno_ccm}")

# For df_crsp2 (CRSP side)
num_nan_permco_crsp = df_crsp2['permco'].isna().sum()
print(f"NaN permcno in df_crsp2: {num_nan_permco_crsp}")

NaN permno in df_ccm1_cleaned: 0
NaN permcno in df_crsp2: 0


In [42]:
unmatched = df_merged[df_merged['permco'].isna()]
print(unmatched[['gvkey', 'permno', 'jdate']].drop_duplicates().head())

      gvkey   permno      jdate
149  001062  26649.0 2012-02-29
150  001062  26649.0 2012-05-31
151  001062  26649.0 2012-08-31
152  001062  26649.0 2012-11-30
153  001062  26649.0 2013-02-28


In [43]:
df_merged_clean = df_merged[df_merged['ME'].notna()]

In [44]:
df_merged_clean.isnull().sum()

gvkey                           0
datadate                        0
year                            0
be                          11518
count                           0
roa                           682
operating_margin             2048
gross_margin                  670
sloan_accruals              67320
revenue_growth               4054
r_and_d_intensity           97679
capex_intensity              2641
roa_stability               39801
revenue_growth_stability    47496
permno                          0
linktype                        0
linkprim                        0
linkdt                          0
linkenddt                       0
jdate                           0
month                           0
quarter                         0
permco                          0
date                            0
shrcd                           0
exchcd                          0
ticker                          0
cusip                           0
ret                             0
retx          

In [45]:
df_merged_clean['ticker'] = df_merged_clean.groupby('permno')['ticker'].transform('first')

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/2246511775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_clean['ticker'] = df_merged_clean.groupby('permno')['ticker'].transform('first')


In [46]:
df_merged_clean = df_merged_clean[df_merged_clean['beme'].notna()]

df_merged_clean

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
0,001004,2012-02-29,2012,890.08,0,0.009306,0.066672,0.198164,<NA>,<NA>,...,AIR,00036110,0.040585,0.040585,888240.15,Q1,2012-Q1,1.040585,1.150235,1.002071
1,001004,2012-05-31,2012,864.649,1,0.005866,0.069975,0.208203,0.067813,0.072354,...,AIR,00036110,-0.220065,-0.220065,485470.4,Q2,2012-Q2,0.779935,0.660273,1.781054
2,001004,2012-08-31,2012,880.6,2,0.008386,0.067212,0.211626,-0.006314,-0.039009,...,AIR,00036110,0.04715,0.04715,597684.96,Q3,2012-Q3,1.04715,1.103857,1.473351
3,001004,2012-11-30,2012,902.8,3,0.00813,0.071373,0.223284,-0.009261,-0.068483,...,AIR,00036110,0.017893,0.017893,613478.4,Q4,2012-Q4,1.017893,0.935445,1.471608
4,001004,2013-02-28,2013,924.0,4,0.0083,0.048827,0.199731,-0.001873,0.014431,...,AIR,00036110,-0.067905,-0.067905,697669.56,Q1,2013-Q1,0.932095,0.940578,1.324409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394126,013353,2016-12-31,2016,3524.2,19,0.00445,0.114264,0.204082,<NA>,-0.105001,...,WEC,92939U10,0.047135,0.047135,18510937.05,Q4,2016-Q4,1.047135,0.97946,0.190385
394127,013353,2017-12-31,2017,3414.3,23,0.005289,0.143434,0.232536,<NA>,-0.003497,...,WEC,92939U10,-0.044035,-0.044035,20963713.68,Q4,2017-Q4,0.955965,1.05814,0.162867
394128,011550,2018-12-31,2018,9819.3,27,0.006133,0.139542,0.244366,<NA>,0.263491,...,WEC,92939U10,-0.044426,-0.044426,21853330.76,Q4,2018-Q4,0.955574,1.037448,0.449327
394129,011550,2019-12-31,2019,10143.8,31,0.006987,0.186444,0.307728,<NA>,0.211132,...,WEC,92939U10,0.040384,0.040384,29092662.28,Q4,2019-Q4,1.040384,0.969821,0.348672


In [47]:
# Count number of unique quarters per permno
permno_quarter_counts = df_merged_clean.groupby('permno')['jdate'].nunique()

# Filter permnos with at least 8 unique quarters (i.e., 2 years)
valid_permnos = permno_quarter_counts[permno_quarter_counts >= 8].index

# Keep only those rows
df_merged_clean = df_merged_clean[df_merged_clean['permno'].isin(valid_permnos)].copy()
df_merged_clean

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
0,001004,2012-02-29,2012,890.08,0,0.009306,0.066672,0.198164,<NA>,<NA>,...,AIR,00036110,0.040585,0.040585,888240.15,Q1,2012-Q1,1.040585,1.150235,1.002071
1,001004,2012-05-31,2012,864.649,1,0.005866,0.069975,0.208203,0.067813,0.072354,...,AIR,00036110,-0.220065,-0.220065,485470.4,Q2,2012-Q2,0.779935,0.660273,1.781054
2,001004,2012-08-31,2012,880.6,2,0.008386,0.067212,0.211626,-0.006314,-0.039009,...,AIR,00036110,0.04715,0.04715,597684.96,Q3,2012-Q3,1.04715,1.103857,1.473351
3,001004,2012-11-30,2012,902.8,3,0.00813,0.071373,0.223284,-0.009261,-0.068483,...,AIR,00036110,0.017893,0.017893,613478.4,Q4,2012-Q4,1.017893,0.935445,1.471608
4,001004,2013-02-28,2013,924.0,4,0.0083,0.048827,0.199731,-0.001873,0.014431,...,AIR,00036110,-0.067905,-0.067905,697669.56,Q1,2013-Q1,0.932095,0.940578,1.324409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394126,013353,2016-12-31,2016,3524.2,19,0.00445,0.114264,0.204082,<NA>,-0.105001,...,WEC,92939U10,0.047135,0.047135,18510937.05,Q4,2016-Q4,1.047135,0.97946,0.190385
394127,013353,2017-12-31,2017,3414.3,23,0.005289,0.143434,0.232536,<NA>,-0.003497,...,WEC,92939U10,-0.044035,-0.044035,20963713.68,Q4,2017-Q4,0.955965,1.05814,0.162867
394128,011550,2018-12-31,2018,9819.3,27,0.006133,0.139542,0.244366,<NA>,0.263491,...,WEC,92939U10,-0.044426,-0.044426,21853330.76,Q4,2018-Q4,0.955574,1.037448,0.449327
394129,011550,2019-12-31,2019,10143.8,31,0.006987,0.186444,0.307728,<NA>,0.211132,...,WEC,92939U10,0.040384,0.040384,29092662.28,Q4,2019-Q4,1.040384,0.969821,0.348672


In [48]:
df_merged_clean.isnull().sum()

gvkey                           0
datadate                        0
year                            0
be                              0
count                           0
roa                           226
operating_margin             1763
gross_margin                  435
sloan_accruals              63197
revenue_growth               3272
r_and_d_intensity           90848
capex_intensity              1889
roa_stability               33931
revenue_growth_stability    41707
permno                          0
linktype                        0
linkprim                        0
linkdt                          0
linkenddt                       0
jdate                           0
month                           0
quarter                         0
permco                          0
date                            0
shrcd                           0
exchcd                          0
ticker                          0
cusip                           0
ret                             0
retx          

### **Handeling missing values**

In [50]:
def fill_missing_custom(df, features, id_col='permno', time_col='jdate'):
    df = df.copy()
    df = df.sort_values([id_col, time_col])

    for col in features:
        print(f"Filling missing for: {col}")

        # Step 1: Interpolate in-between missing values
        df[col] = df.groupby(id_col)[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

        # Step 2: Extrapolate forward using average market change
        # We'll store updated values here and assign all at once
        updates = []

        for firm_id, group in df.groupby(id_col):
            group = group.copy()
            for i in range(1, len(group)):
                if pd.isna(group.iloc[i][col]):
                    prev_val = group.iloc[i - 1][col]
                    prev_date = group.iloc[i - 1][time_col]
                    curr_date = group.iloc[i][time_col]

                    if pd.isna(prev_val):
                        continue

                    # Get others' average change from prev_date → curr_date
                    others_prev = df[(df[time_col] == prev_date) & (df[id_col] != firm_id)][col].dropna()
                    others_curr = df[(df[time_col] == curr_date) & (df[id_col] != firm_id)][col].dropna()

                    if not others_prev.empty and not others_curr.empty:
                        avg_pct_change = (others_curr.mean() - others_prev.mean()) / others_prev.mean()
                        new_val = prev_val * (1 + avg_pct_change)
                        updates.append((group.index[i], new_val))

        # Apply all updates for this column
        for idx, val in updates:
            df.at[idx, col] = val

    return df

knn_features = [
    'roa', 'operating_margin', 'gross_margin',
    'sloan_accruals', 'revenue_growth',
    'r_and_d_intensity', 'capex_intensity',
    'roa_stability', 'revenue_growth_stability'
]

df_filled = fill_missing_custom(df_merged_clean, knn_features)
df_filled

Filling missing for: roa
Filling missing for: operating_margin


/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/3400609669.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  avg_pct_change = (others_curr.mean() - others_prev.mean()) / others_prev.mean()


Filling missing for: gross_margin


/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/3400609669.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  avg_pct_change = (others_curr.mean() - others_prev.mean()) / others_prev.mean()
/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/3400609669.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  avg_pct_change = (others_curr.mean() - others_prev.mean()) / others_prev.mean()


Filling missing for: sloan_accruals
Filling missing for: revenue_growth
Filling missing for: r_and_d_intensity
Filling missing for: capex_intensity
Filling missing for: roa_stability
Filling missing for: revenue_growth_stability


,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
48577,012994,2012-03-31,2012,76.978,0,0.022226,0.177099,0.21503,<NA>,-0.64364,...,EGAS,36720410,0.04876,0.044728,95075.64,Q1,2012-Q1,1.044728,1.021016,0.80965
48578,012994,2012-06-30,2012,75.152,1,-0.004847,0.024761,0.123972,<NA>,-0.64364,...,EGAS,36720410,-0.027325,-0.03164,82375.6,Q2,2012-Q2,0.96836,0.866209,0.912309
48579,012994,2012-09-30,2012,75.626,2,-0.004244,-0.038178,0.066408,<NA>,0.027254,...,EGAS,36720410,0.000501,-0.004008,81080.58,Q3,2012-Q3,0.995992,0.984158,0.932726
48580,012994,2012-12-31,2012,76.344,3,0.010501,0.137991,0.18372,<NA>,1.509989,...,EGAS,36720410,-0.015231,-0.019958,76104.81,Q4,2012-Q4,0.980042,0.938632,1.003143
48581,012994,2013-03-31,2013,80.162,4,0.027454,0.197432,0.230999,<NA>,0.40553,...,EGAS,36720410,0.030684,0.026157,83201.4,Q1,2013-Q1,1.026157,1.093248,0.963469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363468,184996,2023-12-31,2023,62634.0,47,0.074359,0.082012,0.225295,-0.013658,0.077816,...,TSLA,88160R10,0.034988,0.034988,791408800.0,Q4,2023-Q4,1.034988,0.993046,0.079142
363469,184996,2024-03-31,2024,64378.0,48,0.012726,0.054974,0.232008,0.013891,-0.153614,...,TSLA,88160R10,-0.129235,-0.129235,560588301.25034,Q1,2024-Q1,0.870765,0.707462,0.11484
363470,184996,2024-06-30,2024,66468.0,49,0.012408,0.087333,0.229647,-0.01116,0.197127,...,TSLA,88160R10,0.111186,0.111186,632155363.2,Q2,2024-Q2,1.111186,1.125662,0.105145
363471,184996,2024-09-30,2024,69931.0,50,0.018131,0.110079,0.251966,-0.031817,-0.012471,...,TSLA,88160R10,0.221942,0.221942,839047410.0,Q3,2024-Q3,1.221942,1.322164,0.083346


In [51]:
df_filled.isnull().sum()

gvkey                           0
datadate                        0
year                            0
be                              0
count                           0
roa                            38
operating_margin               39
gross_margin                  357
sloan_accruals              41473
revenue_growth                484
r_and_d_intensity           74870
capex_intensity               157
roa_stability                 226
revenue_growth_stability     7863
permno                          0
linktype                        0
linkprim                        0
linkdt                          0
linkenddt                       0
jdate                           0
month                           0
quarter                         0
permco                          0
date                            0
shrcd                           0
exchcd                          0
ticker                          0
cusip                           0
ret                             0
retx          

### **Value and Growth portfolios**

In [52]:
df_final = df_filled[['gvkey', 'permno', 'ticker', 'cumretx_q', 'beme', 'roa', 'operating_margin', 'gross_margin', 
                      'revenue_growth', 'capex_intensity', 
                      'roa_stability', 'revenue_growth_stability', 'year', 'quarter', 'jdate']]

In [53]:
# Rank BEME within each quarter (ascending for growth)
df_final['beme_pct_rank'] = df_final.groupby('quarter')['beme'].rank(pct=True, method='first')

# Filter bottom 30% (growth stocks)
df_growth_portfolio = df_final[df_final['beme_pct_rank'] <= 0.3].copy()

df_growth_portfolio

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4770/1364522422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['beme_pct_rank'] = df_final.groupby('quarter')['beme'].rank(pct=True, method='first')


,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
43464,011903,10025.0,AEPI,1.173712,0.278424,0.000833,0.02162,0.15073,0.108341,0.010262,0.009890,0.071261,2012,Q1,2012-01-31,0.282336
43465,011903,10025.0,AEPI,1.001724,0.290671,0.011594,0.043595,0.164335,0.108341,0.012615,0.009890,0.071261,2012,Q2,2012-04-30,0.297872
43466,011903,10025.0,AEPI,1.07876,0.262526,0.030055,0.085021,0.212875,-0.015759,0.040746,0.009890,0.071261,2012,Q3,2012-07-31,0.260638
43467,011903,10025.0,AEPI,1.055125,0.208511,0.013174,0.043822,0.183367,0.014494,0.036468,0.009890,0.071261,2012,Q4,2012-10-31,0.203865
43468,011903,10025.0,AEPI,1.086949,0.226855,0.015296,0.051587,0.181308,-0.098164,0.032057,0.009890,0.071261,2013,Q1,2013-01-31,0.222212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363468,184996,93436.0,TSLA,0.993046,0.079142,0.074359,0.082012,0.225295,0.077816,0.021638,0.016941,0.118269,2023,Q4,2023-12-31,0.053832
363469,184996,93436.0,TSLA,0.707462,0.11484,0.012726,0.054974,0.232008,-0.153614,0.025424,0.018973,0.138303,2024,Q1,2024-03-31,0.087505
363470,184996,93436.0,TSLA,1.125662,0.105145,0.012408,0.087333,0.229647,0.197127,0.020136,0.020744,0.140593,2024,Q2,2024-06-30,0.079868
363471,184996,93436.0,TSLA,1.322164,0.083346,0.018131,0.110079,0.251966,-0.012471,0.029311,0.020845,0.114321,2024,Q3,2024-09-30,0.054858


In [54]:
# 📌 Value Portfolio (Top 30% by BEME → High BEME ratios)
df_value_portfolio = df_final[df_final['beme_pct_rank'] >= 0.7].copy()

df_value_portfolio

,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
48577,012994,10001.0,EGAS,1.021016,0.80965,0.022226,0.177099,0.21503,-0.64364,0.029558,0.013291,0.886517,2012,Q1,2012-03-31,0.727562
48578,012994,10001.0,EGAS,0.866209,0.912309,-0.004847,0.024761,0.123972,-0.64364,0.029421,0.013291,0.886517,2012,Q2,2012-06-30,0.777105
48579,012994,10001.0,EGAS,0.984158,0.932726,-0.004244,-0.038178,0.066408,0.027254,0.02729,0.013291,0.886517,2012,Q3,2012-09-30,0.782691
48580,012994,10001.0,EGAS,0.938632,1.003143,0.010501,0.137991,0.18372,1.509989,0.030803,0.013291,0.886517,2012,Q4,2012-12-31,0.825297
48581,012994,10001.0,EGAS,1.093248,0.963469,0.027454,0.197432,0.230999,0.40553,0.019604,0.013291,0.886517,2013,Q1,2013-03-31,0.802672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358767,184259,93434.0,SANW,0.624999,1.872698,-0.045019,-0.253086,0.42762,-0.231942,0.005826,0.091510,0.446597,2023,Q4,2023-12-31,0.957838
358768,184259,93434.0,SANW,0.691,2.441357,-0.041296,-0.129768,0.274543,1.217986,0.001058,0.091341,0.552817,2024,Q1,2024-03-31,0.975781
358769,184259,93434.0,SANW,0.744263,2.513548,-0.100326,-0.262618,0.167274,-0.191269,0.002692,0.091833,0.556151,2024,Q2,2024-06-30,0.977301
358770,184259,93434.0,SANW,0.513889,3.798039,-0.167993,-0.47948,0.16079,-0.439339,0.001428,0.106133,0.585176,2024,Q3,2024-09-30,0.989293


In [55]:
df_growth_portfolio.isnull().sum()

gvkey                          0
permno                         0
ticker                         0
cumretx_q                      0
beme                           0
roa                            0
operating_margin               1
gross_margin                 111
revenue_growth               189
capex_intensity                0
roa_stability                 72
revenue_growth_stability    2738
year                           0
quarter                        0
jdate                          0
beme_pct_rank                  0
dtype: int64

In [56]:
df_value_portfolio.isnull().sum()

gvkey                          0
permno                         0
ticker                         0
cumretx_q                      0
beme                           0
roa                           38
operating_margin              38
gross_margin                 140
revenue_growth               131
capex_intensity              132
roa_stability                112
revenue_growth_stability    2281
year                           0
quarter                        0
jdate                          0
beme_pct_rank                  0
dtype: int64

In [57]:
df_g_portfolio = df_growth_portfolio.dropna(subset=[
    'operating_margin',
    'gross_margin',
    'revenue_growth',
    'capex_intensity',
    'roa_stability',
    'revenue_growth_stability'
])

In [58]:
df_v_portfolio = df_value_portfolio.dropna(subset=[
    'operating_margin',
    'gross_margin',
    'revenue_growth',
    'capex_intensity',
    'roa_stability',
    'revenue_growth_stability'
])

In [59]:
# Create quarterly summary with unique permno count and total permno count per quarter
quarterly_summary_growth = (
    df_g_portfolio
    .groupby(['year', 'quarter'])
    .agg(
        unique_permnos=('permno', 'nunique'),
        total_permnos=('permno', 'count')
    )
    .reset_index()
    .sort_values(['year', 'quarter'])
)

quarterly_summary_growth.head()

,year,quarter,unique_permnos,total_permnos
0,2012,Q1,668,668
1,2012,Q2,631,631
2,2012,Q3,670,670
3,2012,Q4,664,664
4,2013,Q1,773,773


In [60]:
# Create quarterly summary with unique permno count and total permno count per quarter
quarterly_summary_value = (
    df_v_portfolio
    .groupby(['year', 'quarter'])
    .agg(
        unique_permnos=('permno', 'nunique'),
        total_permnos=('permno', 'count')
    )
    .reset_index()
    .sort_values(['year', 'quarter'])
)

quarterly_summary_value.head()

,year,quarter,unique_permnos,total_permnos
0,2012,Q1,1106,1106
1,2012,Q2,1189,1189
2,2012,Q3,1161,1161
3,2012,Q4,1205,1205
4,2013,Q1,1030,1030


In [61]:
quarterly_summary_value.tail()

,year,quarter,unique_permnos,total_permnos
47,2023,Q4,1089,1089
48,2024,Q1,1011,1011
49,2024,Q2,1079,1079
50,2024,Q3,987,987
51,2024,Q4,936,936


In [62]:
df_final_growth = df_g_portfolio.sort_values(['jdate', 'permno'])
df_final_growth

,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
43464,011903,10025.0,AEPI,1.173712,0.278424,0.000833,0.02162,0.15073,0.108341,0.010262,0.009890,0.071261,2012,Q1,2012-01-31,0.282336
364295,185190,12080.0,GMAN,1.155131,0.277119,0.062769,0.087199,0.397523,-0.273248,0.061144,0.021228,0.261435,2012,Q1,2012-01-31,0.280664
54444,014603,12266.0,PPHM,0.951553,0.141359,-0.376698,-3.380981,0.315453,-0.370619,0.015958,0.101901,0.540270,2012,Q1,2012-01-31,0.118897
364412,185195,12363.0,VRA,1.110388,0.085489,0.091575,0.229863,0.582374,-0.128714,0.052491,0.020557,0.155336,2012,Q1,2012-01-31,0.057283
362726,184807,12410.0,TFM,1.079198,0.061342,0.056467,0.091205,0.337056,0.012507,0.05272,0.014122,0.078523,2012,Q1,2012-01-31,0.036638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,001414,93330.0,PRI,1.023647,0.24941,0.011455,0.284959,0.290445,0.018074,0.000177,0.003658,0.022809,2024,Q4,2024-12-31,0.253181
343253,179841,93345.0,CDXS,1.548703,0.172415,-0.069632,-0.342078,0.82973,0.672251,0.011905,0.062299,0.876491,2024,Q4,2024-12-31,0.159571
356686,183974,93356.0,SPSC,0.947571,0.123651,0.017027,0.14478,0.702329,0.044115,0.006026,0.001995,0.015774,2024,Q4,2024-12-31,0.102162
356343,183945,93371.0,CRMD,1.002476,0.172248,0.113289,0.448254,0.966581,1.724337,0.000084,0.118385,1.954662,2024,Q4,2024-12-31,0.159319


In [63]:
df_final_value = df_v_portfolio.sort_values(['jdate', 'permno'])
df_final_value

,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
44500,012171,10182.0,TECD,1.050799,0.925609,0.009346,0.015764,0.05476,-0.169066,0.000761,0.002637,0.156836,2012,Q1,2012-01-31,0.786335
44658,012234,10259.0,SIGM,1.005,1.252324,-0.06338,-0.485182,0.613817,0.131924,0.009412,0.056628,0.316918,2012,Q1,2012-01-31,0.891483
45489,012421,10514.0,TWMC,0.976323,2.083125,0.052689,0.089831,0.359138,-0.418524,0.000667,0.043084,0.472293,2012,Q1,2012-01-31,0.966226
46481,012597,10645.0,LAKE,1.01454,1.488577,-0.015116,-0.049988,0.299777,0.189239,0.003586,0.084157,0.096716,2012,Q1,2012-01-31,0.927044
5818,002436,10866.0,BWS,1.061798,1.040102,-0.006658,0.016903,0.405732,-0.048835,0.006143,0.012738,0.128493,2012,Q1,2012-01-31,0.833444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358164,184167,93304.0,FIBK,1.058344,0.973457,0.001788,0.281116,0.610515,0.00269,0.000227,0.000221,0.035439,2024,Q4,2024-12-31,0.811954
261710,133764,93368.0,EBMT,0.970869,1.422003,0.001632,0.232605,0.684539,-0.015247,0.00158,0.000310,0.066104,2024,Q4,2024-12-31,0.923012
363033,184899,93374.0,FAF,0.945917,0.763188,0.004856,0.085217,0.116076,0.198421,0.003917,0.004712,0.132759,2024,Q4,2024-12-31,0.703441
364115,185138,93426.0,VPG,0.906178,1.122973,0.001703,0.007956,0.438867,-0.040593,0.004874,0.006729,0.047183,2024,Q4,2024-12-31,0.866314


### **Combine value and growth portfolios and save**

In [64]:
df_final_growth['portfolio_type'] = 'Growth'
df_final_value['portfolio_type'] = 'Value'

# Concatenate both datasets
df_combined_portfolio = pd.concat([df_final_growth, df_final_value], ignore_index=True)

In [65]:
# Save to CSV
df_combined_portfolio.to_csv('stocks_portfolio_overall.csv', index=False)

In [66]:
df_final_value.isnull().sum()

gvkey                       0
permno                      0
ticker                      0
cumretx_q                   0
beme                        0
roa                         0
operating_margin            0
gross_margin                0
revenue_growth              0
capex_intensity             0
roa_stability               0
revenue_growth_stability    0
year                        0
quarter                     0
jdate                       0
beme_pct_rank               0
portfolio_type              0
dtype: int64